In [59]:
import torch
from torch.autograd import Variable
dtype = torch.FloatTensor

In [60]:
import numpy as np
pi = np.pi
%matplotlib notebook
import matplotlib.pyplot as plt

## Simultaneous search for policy and verifying Lyapunov function

$$\dot{x} = f(x,u)$$

$$ u = \pi_\theta(x)) $$

$$ V(x) = p.s.d. \text{by construction, but parameterized by parameters } \psi $$

$$ \dot{V} = \frac{dV}{dx} \dot{x}$$
$$ = \big[ \frac{dV}{dx}\big]^T \big[f(x,\pi_{\theta}(x) \big] $$

### Loss function

$X$ = {$x_1, x_2, ..., x_N$} many samples

$$ L(\theta) = \sum_{i} l(x_i, \theta) $$

\begin{equation}
  \mathcal{l}(x_i,\theta) =
  \begin{cases}
    \dot{V}(x_i, \theta) & \text{if $\dot{V}(x_i, \theta) > 0$} \\
    0 & \text{otherwise}
  \end{cases}
\end{equation}



In [61]:
l = 1.0   # length in pixels
g = 1.0    # gravity in m/s**2
m = 1.0    # mass in kg
b = 1.0    # damping
u_constraint = 0.5 # input constraint

x_i = Variable(torch.FloatTensor([1.0, 1.1]).type(dtype), requires_grad=True)
print x_i

P_size = 3
P = Variable(torch.FloatTensor([1, 1, 1]).type(dtype), requires_grad=True)
print P
### Step 1: compute V(x)
def compute_V(x):
    '''V(x) = x_1^2 + x_2^2'''
    monomials = Variable(torch.zeros(P_size))
    monomials[0] = x[1].pow(2)
    monomials[1] = torch.cos(x[0])
    monomials[2] = 1
    polynomial = torch.dot(P, monomials)
    return polynomial.pow(2)

def compute_V_energy_based(x):
    '''V(x) = (E - E_d)^2'''
    energy = 0.5*m*l**2*x[1]**2 + m*g*l*torch.cos(x[0])
    energy_desired = m*g*l
    difference_squared = (energy-energy_desired).pow(2)
    return difference_squared

print x_i.shape
print "V is", compute_V(x_i)
x_i_2 = Variable(torch.FloatTensor([1.1, 2.2]).type(dtype), requires_grad=True)
print "V is", compute_V(x_i_2)

many_x_i = Variable(torch.FloatTensor([[1.0, 1.1, 2.0],[1.1, 2.2, 3.0]]).type(dtype), requires_grad=True)

def compute_V_vectorized(x):
    monomials = Variable(torch.zeros(K_size, x.shape[1]))
    monomials[0] = x[1,:].pow(2)
    monomials[1] = torch.cos(x[0,:])
    monomials[2] = 1
    P_transpose = torch.t(P.unsqueeze(1))
    polynomial = P_transpose.mm(monomials)
    return polynomial.pow(2)

def compute_V_energy_based_vectorized(x):
    energy_vectorized = 0.5*m*l**2*x[1,:].pow(2) + m*g*l*torch.cos(x[0,:])
    energy_desired = m*g*l
    difference_squared = (energy_vectorized-energy_desired).pow(2)
    return difference_squared
    
print many_x_i
print compute_V_energy_based_vectorized(many_x_i)

Variable containing:
 1.0000
 1.1000
[torch.FloatTensor of size 2]

Variable containing:
 1
 1
 1
[torch.FloatTensor of size 3]

torch.Size([2])
V is Variable containing:
 7.5642
[torch.FloatTensor of size 1]

V is Variable containing:
 39.6094
[torch.FloatTensor of size 1]

Variable containing:
 1.0000  1.1000  2.0000
 1.1000  2.2000  3.0000
[torch.FloatTensor of size 2x3]

Variable containing:
 0.0211
 3.5104
 9.5102
[torch.FloatTensor of size 3]



In [63]:
### Step 2: compute dV/dx

### Step 3: initialize policy parameters
# K = Variable(torch.FloatTensor([1, 2]).type(dtype), requires_grad=True)
# print K

### Step 4: define dynamics function
def dynamics(x):
    xdot = Variable(torch.zeros(2).type(dtype))
    xdot[0] = x[1]
    xdot[1] = -torch.dot(K,x)
    return xdot

def pendulum_policy(x):
    x_desired = Variable(torch.FloatTensor([0, 0]).type(dtype), requires_grad=False)
    return -torch.dot(K,x-x_desired)

def pendulum_energy_policy(x):
    energy = 0.5*m*l**2*x[1]**2 + m*g*l*torch.cos(x[0])
    energy_desired = m*g*l
    return -1*x[1]*(energy - energy_desired)

K_size = 3
#K = torch.randn(K_size)
#K = Variable(K, requires_grad=True)
K = Variable(torch.FloatTensor([0.5*m*l**2, m*g*l, -m*g*l]).type(dtype), requires_grad=True)
def pendulum_policy_search(x):
    monomials = Variable(torch.zeros(K_size))
    monomials[0] = x[1].pow(3)
    monomials[1] = x[1]*torch.cos(x[0])
    monomials[2] = x[1]
    u = -torch.dot(K, monomials)
    return u.clamp(min=u_constraint).clamp(max=u_constraint)

def pendulum_dynamics(x):
    xdot = Variable(torch.zeros(2).type(dtype))
    xdot[0] = x[1]
    u = pendulum_policy_search(x)
    xdot[1] = (m * g * l * torch.sin(x[0:1])) + u
    return xdot
    
xdot = pendulum_dynamics(x_i)
print xdot
print pendulum_dynamics(x_i_2)
print "policy search is", pendulum_policy_search(x_i)
print "policy search is", pendulum_policy_search(x_i_2)

many_x_i = Variable(torch.FloatTensor([[1.0, 1.1, 2.0],[1.1, 2.2, 3.0]]).type(dtype), requires_grad=True)

def dynamics_vectorized(x):
    xdot = Variable(torch.zeros(2,x.shape[1]).type(dtype))
    xdot[0,:] = x[1,:]
    K_transpose = torch.t(K.unsqueeze(1))
    xdot[1,:] = -K_transpose.mm(x)
    return xdot

def pendulum_policy_vectorized(x):
    x_desired = Variable(torch.FloatTensor([0, 0]).type(dtype), requires_grad=False)
    x_desired_vectorized = torch.t(x_desired.repeat(x.shape[1], 1))
    K_transpose = torch.t(K.unsqueeze(1))
    return K_transpose.mm(x - x_desired_vectorized)

def pendulum_energy_policy_vectorized(x):
    energy_vectorized = 0.5*m*l**2*x[1,:]**2 + m*g*l*torch.cos(x[0,:])
    energy_desired = m*g*l
    return -K[0]*x[1,:]*(energy_vectorized - energy_desired)

def pendulum_policy_search_vectorized(x):
    monomials = Variable(torch.zeros(K_size, x.shape[1]))
    monomials[0] = x[1,:].pow(3)
    monomials[1] = x[1,:]*torch.cos(x[0,:])
    monomials[2] = x[1,:]
    K_transpose = torch.t(K.unsqueeze(1))
    return (-K_transpose.mm(monomials)).clamp(min=-u_constraint).clamp(max=u_constraint)

def pendulum_dynamics_vectorized(x):
    xdot = Variable(torch.zeros(2,x.shape[1]).type(dtype))
    xdot[0,:] = x[1,:]
    u = pendulum_policy_search_vectorized(x)
    xdot[1,:] = (m * g * l * torch.sin(x[0:1])) + u
    return xdot
      
print many_x_i
print pendulum_dynamics_vectorized(many_x_i)
print "policy search is", pendulum_policy_search_vectorized(many_x_i)

Variable containing:
 1.1000
 1.3415
[torch.FloatTensor of size 2]

Variable containing:
 2.2000
 1.3912
[torch.FloatTensor of size 2]

policy search is Variable containing:
 0.5000
[torch.FloatTensor of size 1]

policy search is Variable containing:
 0.5000
[torch.FloatTensor of size 1]

Variable containing:
 1.0000  1.1000  2.0000
 1.1000  2.2000  3.0000
[torch.FloatTensor of size 2x3]

Variable containing:
 1.1000  2.2000  3.0000
 0.6816  0.3912  0.4093
[torch.FloatTensor of size 2x3]

policy search is Variable containing:
-0.1598 -0.5000 -0.5000
[torch.FloatTensor of size 1x3]



In [64]:
## COMPARE ENERGY POLICY

print "energy computes", pendulum_energy_policy(x_i)
print "search computes", pendulum_policy_search(x_i)

energy computes Variable containing:
-0.1598
[torch.FloatTensor of size 1]

search computes Variable containing:
 0.5000
[torch.FloatTensor of size 1]



In [65]:
def compute_Vdot(x):
    V = compute_V(x)
#     print x.shape
#     print V.shape
#     print V
    V.backward(torch.ones(1).type(dtype),retain_graph=True)
#     print "x.grad is", x.grad
    jacobian_x = Variable(x.grad.data)
#     print "jacobian_x is", jacobian_x
    f = pendulum_dynamics(x)
    Vdot = torch.dot(jacobian_x,f)
    return Vdot

print "x_i is", x_i
if x_i.grad is not None:
    print x_i.grad.zero_()
Vdot = compute_Vdot(x_i)
print "Vdot is", Vdot
Vdot.backward()

def compute_Vdot_vectorized(x):
    V = compute_V_vectorized(x)
#     print V
#     print x.shape
#     print "x.grad is", x.grad
#    V.backward(torch.ones(1,1).type(dtype),retain_graph=True)
#     print x
    V.backward(torch.ones(1,x.shape[1]).type(dtype),retain_graph=True)
    jacobian_x = Variable(x.grad.data)
#     print "jacobian_x is", jacobian_x
    f = pendulum_dynamics_vectorized(x)
    Vdot = (jacobian_x*f).sum(dim=0)
    return Vdot

print "many_x_i is", many_x_i

Vdot_vec = compute_Vdot_vectorized(many_x_i)
print "Vdot_vec is", Vdot_vec
#Vdot.backward(torch.ones(many_x_i.shape[1]).type(dtype))
cost = Vdot_vec.clamp(min=0).sum()
print cost
cost.backward()
many_x_i.grad.data.zero_()



x_i is Variable containing:
 1.0000
 1.1000
[torch.FloatTensor of size 2]

Vdot is Variable containing:
 11.1421
[torch.FloatTensor of size 1]

many_x_i is Variable containing:
 1.0000  1.1000  2.0000
 1.1000  2.2000  3.0000
[torch.FloatTensor of size 2x3]

Vdot_vec is Variable containing:
 3.1573
-3.0127
-5.2157
[torch.FloatTensor of size 3]

Variable containing:
 3.1573
[torch.FloatTensor of size 1]




 0  0  0
 0  0  0
[torch.FloatTensor of size 2x3]

## First let's verify, no synthesis
 
K = [1, 2] and P = [1, 1] should be stable

In [66]:
#K = Variable(torch.FloatTensor([1, 2]).type(dtype), requires_grad=True)
#P = Variable(torch.FloatTensor([1, 1]).type(dtype), requires_grad=True)
for i in range(10000):
    x_i = Variable(torch.randn(2).type(dtype), requires_grad=True)
    Vdot = compute_Vdot(x_i)
    if Vdot.data[0] > 0:
        print "false, counterexample found: ", x_i
        print i
        print compute_V(x_i)
        print Vdot.data[0]
        break

false, counterexample found:  Variable containing:
 0.7921
 0.2047
[torch.FloatTensor of size 2]

0
Variable containing:
 3.0425
[torch.FloatTensor of size 1]

1.22249996662


## Now let's search for K, from an initialization not stable

In [67]:
P = Variable(torch.rand(3).type(dtype), requires_grad=True)
K = Variable(torch.randn(3).type(dtype), requires_grad=True)

print "Initial P", P
print "Initial K", K


for i in range(10000):
    x_i = Variable(torch.randn(2).type(dtype), requires_grad=True)
    Vdot = compute_Vdot(x_i)
    if Vdot.data[0] > 0:
        print "false, counterexample found: ", x_i
        break

Initial P Variable containing:
 0.2102
 0.1877
 0.2417
[torch.FloatTensor of size 3]

Initial K Variable containing:
-0.6425
-0.1031
 0.2404
[torch.FloatTensor of size 3]

false, counterexample found:  Variable containing:
 0.0509
 1.5156
[torch.FloatTensor of size 2]



In [99]:
# Reset 
P = Variable(torch.rand(3).type(dtype), requires_grad=True)
K = Variable(torch.randn(3).type(dtype), requires_grad=True)

In [100]:
P.data = P.data/(abs(P.data.sum()))

In [107]:
print P
print K

Variable containing:
1.00000e-04 *
 -0.4910
 -8.6998
  6.6812
[torch.FloatTensor of size 3]

Variable containing:
 0.9489
-0.4682
-0.5841
[torch.FloatTensor of size 3]



In [108]:
## optimization plotting tool

cost_current_iteration = 0
cost_history = []
cost_iteration_number_history = []

f, (cost_axis) = plt.subplots(1, 1)

cost_axis.plot(cost_iteration_number_history, cost_history)
cost_axis.set_title('Running cost')

plt.tight_layout()

<IPython.core.display.Javascript object>

In [109]:
## optimize

num_iterations = 2000
num_samples_per_iteration = 1000
step_rate = 1e-5
slack_var = .01

# K has already been initialized above, and initial policy visualized

import time
print "first P is", P
print "first K is", K

for cost_iteration in range(num_iterations):
    
    start = time.time()
    
    cost = 0
    
    samples = torch.randn(2,num_samples_per_iteration).type(dtype)
    many_x_i = Variable(samples, requires_grad=True)
    Vdot = compute_Vdot_vectorized(many_x_i)
    cost = Vdot.clamp(min=-slack_var).sum()
        
    ## Automatically differentiate
    cost.backward()

    # Update K via gradient descent
    K.data -= step_rate * K.grad.data
    P.data -= step_rate * P.grad.data
    
    # Project P into feasible
    #P.data = torch.abs(P.data)
    # Make P numerically stable by normalizing
    P.data = P.data/(abs(P.data.sum()))
      
    # Manually zero the gradients after running the backward pass
    K.grad.data.zero_()
    P.grad.data.zero_()
    
    print time.time() - start, "is time for one step of grad descent"
    print 
    print cost.data[0]
    
    # handle plotting
    cost_history.append(cost.data[0])
    cost_iteration_number_history.append(cost_iteration)
    
    if cost_iteration % 10 == 0:
        cost_axis.lines[0].set_xdata(cost_iteration_number_history)
        cost_axis.lines[0].set_ydata(cost_history)
        cost_axis.relim()
        cost_axis.autoscale_view()
        cost_axis.figure.canvas.draw()
        
    if cost.data[0] == 0:
        break
        
    print "P is", P
    print "K is", K
    
    if cost_iteration == num_iterations - 1:
        print "didn't converge"

first P is Variable containing:
1.00000e-04 *
 -0.4910
 -8.6998
  6.6812
[torch.FloatTensor of size 3]

first K is Variable containing:
 0.9489
-0.4682
-0.5841
[torch.FloatTensor of size 3]

0.00284504890442 is time for one step of grad descent

-2.70008308689e-06
P is Variable containing:
-0.1931
-3.4662
 2.6593
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9489
-0.4682
-0.5841
[torch.FloatTensor of size 3]

0.00274395942688 is time for one step of grad descent

2107.2019043
P is Variable containing:
-0.1948
-3.4588
 2.6536
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9490
-0.4677
-0.5841
[torch.FloatTensor of size 3]

0.00243496894836 is time for one step of grad descent

1633.59533691
P is Variable containing:
-0.1938
-3.4704
 2.6642
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9491
-0.4672
-0.5842
[torch.FloatTensor of size 3]

0.00228095054626 is time for one step of grad descent

2134.80786133
P is Variable containing:
-0.1953
-3.4579
 

P is Variable containing:
-0.1961
-3.4568
 2.6529
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9498
-0.4440
-0.5926
[torch.FloatTensor of size 3]

0.00188493728638 is time for one step of grad descent

1738.8458252
P is Variable containing:
-0.1965
-3.4586
 2.6551
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9500
-0.4436
-0.5926
[torch.FloatTensor of size 3]

0.00148916244507 is time for one step of grad descent

1516.8828125
P is Variable containing:
-0.1966
-3.4581
 2.6548
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9500
-0.4431
-0.5927
[torch.FloatTensor of size 3]

0.00143194198608 is time for one step of grad descent

1847.49938965
P is Variable containing:
-0.1973
-3.4583
 2.6556
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9500
-0.4426
-0.5929
[torch.FloatTensor of size 3]

0.00133085250854 is time for one step of grad descent

1998.27636719
P is Variable containing:
-0.1976
-3.4600
 2.6576
[torch.FloatTensor of size 3

K is Variable containing:
 0.9482
-0.4113
-0.6062
[torch.FloatTensor of size 3]

0.00201511383057 is time for one step of grad descent

1970.41918945
P is Variable containing:
-0.1958
-3.4807
 2.6766
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9482
-0.4108
-0.6065
[torch.FloatTensor of size 3]

0.00177597999573 is time for one step of grad descent

1809.10351562
P is Variable containing:
-0.1968
-3.4736
 2.6704
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9481
-0.4104
-0.6067
[torch.FloatTensor of size 3]

0.00185799598694 is time for one step of grad descent

1839.8079834
P is Variable containing:
-0.1955
-3.4794
 2.6749
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9481
-0.4099
-0.6070
[torch.FloatTensor of size 3]

0.00168609619141 is time for one step of grad descent

2012.00463867
P is Variable containing:
-0.1956
-3.4790
 2.6746
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9480
-0.4095
-0.6073
[torch.FloatTensor of size 

0.00369501113892 is time for one step of grad descent

1904.51940918
P is Variable containing:
-0.1939
-3.4548
 2.6486
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9455
-0.3825
-0.6199
[torch.FloatTensor of size 3]

0.00225782394409 is time for one step of grad descent

2011.1307373
P is Variable containing:
-0.1944
-3.4503
 2.6448
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9453
-0.3819
-0.6203
[torch.FloatTensor of size 3]

0.00213003158569 is time for one step of grad descent

1985.12988281
P is Variable containing:
-0.1946
-3.4427
 2.6373
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9451
-0.3815
-0.6207
[torch.FloatTensor of size 3]

0.00218105316162 is time for one step of grad descent

1851.94763184
P is Variable containing:
-0.1938
-3.4482
 2.6421
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9448
-0.3811
-0.6212
[torch.FloatTensor of size 3]

0.00197792053223 is time for one step of grad descent

1406.66394043
P is Var

P is Variable containing:
-0.1890
-3.4998
 2.6888
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9415
-0.3515
-0.6352
[torch.FloatTensor of size 3]

0.00159907341003 is time for one step of grad descent

1943.73779297
P is Variable containing:
-0.1904
-3.4924
 2.6827
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9413
-0.3509
-0.6356
[torch.FloatTensor of size 3]

0.00145196914673 is time for one step of grad descent

1879.42248535
P is Variable containing:
-0.1902
-3.4901
 2.6803
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9413
-0.3504
-0.6358
[torch.FloatTensor of size 3]

0.00144505500793 is time for one step of grad descent

1866.0435791
P is Variable containing:
-0.1897
-3.4892
 2.6789
[torch.FloatTensor of size 3]

K is Variable containing:
 0.9413
-0.3500
-0.6359
[torch.FloatTensor of size 3]

0.00150108337402 is time for one step of grad descent

1819.5904541
P is Variable containing:
-0.1893
-3.4852
 2.6745
[torch.FloatTensor of size 3

...

0.00251388549805 is time for one step of grad descent

1821.3125
P is Variable containing:
-0.1864
-3.5009
 2.6874
[torch.FloatTensor of size 3]

K is Variable containing:
 0.3051
 0.6237
-1.0788
[torch.FloatTensor of size 3]

0.00176405906677 is time for one step of grad descent

1893.39489746
P is Variable containing:
-0.1881
-3.4933
 2.6814
[torch.FloatTensor of size 3]

K is Variable containing:
 0.3055
 0.6245
-1.0784
[torch.FloatTensor of size 3]

0.00154399871826 is time for one step of grad descent

1716.47485352
P is Variable containing:
-0.1902
-3.4845
 2.6747
[torch.FloatTensor of size 3]

K is Variable containing:
 0.3045
 0.6253
-1.0783
[torch.FloatTensor of size 3]

0.00148987770081 is time for one step of grad descent

1843.23168945
P is Variable containing:
-0.1907
-3.4810
 2.6717
[torch.FloatTensor of size 3]

K is Variable containing:
 0.3040
 0.6261
-1.0781
[torch.FloatTensor of size 3]

0.00147986412048 is time for one step of grad descent

1635.12268066
P is Variab

P is Variable containing:
-0.1908
-3.4662
 2.6570
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2876
 0.6613
-1.0677
[torch.FloatTensor of size 3]

0.00230383872986 is time for one step of grad descent

1809.33605957
P is Variable containing:
-0.1908
-3.4635
 2.6543
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2877
 0.6620
-1.0674
[torch.FloatTensor of size 3]

0.0020649433136 is time for one step of grad descent

1814.42651367
P is Variable containing:
-0.1902
-3.4699
 2.6601
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2868
 0.6629
-1.0673
[torch.FloatTensor of size 3]

0.00160193443298 is time for one step of grad descent

1430.90368652
P is Variable containing:
-0.1902
-3.4664
 2.6567
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2873
 0.6635
-1.0668
[torch.FloatTensor of size 3]

0.00172400474548 is time for one step of grad descent

1812.31176758
P is Variable containing:
-0.1902
-3.4655
 2.6557
[torch.FloatTensor of size 

P is Variable containing:
-0.1963
-3.4681
 2.6644
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2653
 0.7064
-1.0532
[torch.FloatTensor of size 3]

0.00172305107117 is time for one step of grad descent

1539.45776367
P is Variable containing:
-0.1955
-3.4772
 2.6727
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2651
 0.7070
-1.0530
[torch.FloatTensor of size 3]

0.00240397453308 is time for one step of grad descent

1709.96704102
P is Variable containing:
-0.1931
-3.4888
 2.6818
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2652
 0.7077
-1.0526
[torch.FloatTensor of size 3]

0.00261807441711 is time for one step of grad descent

1909.54846191
P is Variable containing:
-0.1969
-3.4680
 2.6649
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2650
 0.7085
-1.0524
[torch.FloatTensor of size 3]

0.00267887115479 is time for one step of grad descent

1744.35705566
P is Variable containing:
-0.1966
-3.4696
 2.6662
[torch.FloatTensor of size

P is Variable containing:
-0.1932
-3.4923
 2.6855
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2480
 0.7428
-1.0411
[torch.FloatTensor of size 3]

0.00173711776733 is time for one step of grad descent

1890.98571777
P is Variable containing:
-0.1943
-3.4847
 2.6790
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2469
 0.7436
-1.0409
[torch.FloatTensor of size 3]

0.00148010253906 is time for one step of grad descent

1634.12597656
P is Variable containing:
-0.1925
-3.4931
 2.6857
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2467
 0.7444
-1.0405
[torch.FloatTensor of size 3]

0.00214099884033 is time for one step of grad descent

1941.23937988
P is Variable containing:
-0.1905
-3.5071
 2.6976
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2455
 0.7452
-1.0404
[torch.FloatTensor of size 3]

0.00209784507751 is time for one step of grad descent

1676.81982422
P is Variable containing:
-0.1917
-3.4983
 2.6900
[torch.FloatTensor of size

0.00861811637878 is time for one step of grad descent

1887.23852539
P is Variable containing:
-0.1958
-3.4639
 2.6597
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2311
 0.7810
-1.0272
[torch.FloatTensor of size 3]

0.00299310684204 is time for one step of grad descent

1793.46398926
P is Variable containing:
-0.1974
-3.4501
 2.6475
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2306
 0.7817
-1.0269
[torch.FloatTensor of size 3]

0.00283408164978 is time for one step of grad descent

1599.90380859
P is Variable containing:
-0.1966
-3.4547
 2.6512
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2303
 0.7824
-1.0267
[torch.FloatTensor of size 3]

0.00261282920837 is time for one step of grad descent

1799.0447998
P is Variable containing:
-0.1964
-3.4496
 2.6460
[torch.FloatTensor of size 3]

K is Variable containing:
 0.2297
 0.7832
-1.0265
[torch.FloatTensor of size 3]

0.00260877609253 is time for one step of grad descent

1830.96850586
P is Var

In [110]:
print K
print P

Variable containing:
 0.2293
 0.7855
-1.0254
[torch.FloatTensor of size 3]

Variable containing:
-0.1967
-3.4499
 2.6466
[torch.FloatTensor of size 3]



In [111]:
# THIS IS THE ANSWER
#K = Variable(torch.FloatTensor([0.5*m*l**2, m*g*l, -m*g*l]).type(dtype), requires_grad=True)
for i in range(10000):
    x_i = Variable(torch.randn(2).type(dtype), requires_grad=True)
    Vdot = compute_Vdot(x_i)
    if Vdot.data[0] > 0:
        print "false, counterexample found: ", x_i
        print i
        print Vdot.data[0]
        break

false, counterexample found:  Variable containing:
 0.5139
-0.6383
[torch.FloatTensor of size 2]

3
0.72987985611


In [21]:
# Note: first time I ended up with these values:

# print K
# print P
# Variable containing:
#   0.2410
#  16.2734
# [torch.FloatTensor of size 2]

# Variable containing:
#  0.3484
#  0.8831
# [torch.FloatTensor of size 2]

#Which gave only:

# # condition (a):  p_1 - p_2*k_1
# print P[0] - P[1]*K[0]

# # condition (b): k_2 > 0
# print K[1]

# Variable containing:
#  0.1356
# [torch.FloatTensor of size 1]

# Variable containing:
#  16.2734
# [torch.FloatTensor of size 1]

# But this was Vdot negative semidefinite over all samples, since k_2 was so large it could outweigh the first term